# Tensorflow

**Estos son ejemplos del libro Hands-On Machine Learning with Scikit-Learn and Tensorflow de Aurelién Gerón**

Tensorflow es el API de google para Machine Learning y obviamente para Deep Learning que es un subset de funciones de Machine Learning. 

Aquí utilizaremos Tensorflow 1.x y en otros laboratorios observaremos la diferencia entre Tensorflow 1.x y Tensorflow 2.0 que elimina por ejemplo el uso de sesiones y de eager excecution.

Para observar el entrenamiento, Tensorflow tiene una herramienta 'live' llamada Tensorboard que sirve para visualizar el training y así tomar decisiones si estamos haciendo un buen 'training' de la data y por consiguiente un buen modelo.

Otra ventaja de utilizar Tensorflow es que puedes realizar entrenamiento tanto en el CPU como el GPU, soporta computación distribuida entre comptuadores y servidores.

Podemos hacer entrenamiento de millones de características en millones de instancias, cabe destacar que esta herramienta inicialmente desarrollada por Google Brain es la herramienta que utilizan para producción los ingenieros de google para aplicaciones como Google Cloud, Speech, Photos, Search y también para otras como YouTube.

Algunas cosas a destacar de esta herramienta son:
- Soporta varios sistemas como Windows, Linux y MacOS
- API de python simplista compatible con TFLearn, que puede involucrar a Scikit-Learn (versión 1.x no es tan pythonic, más si al versión 2.0)
- TF-slim para simplificar el pipeline de ML
- Keras, que es un framework simplista para realizar modelos de redes neuronales
- Implementación optimizada de algoritmos en C++ para python como wrapper
- Nodos de optimización avanzada 
- Tensorboard, que es la herramienta de visualización de training y gráficos del framework
- Servicio de nube Tensorflow Graphs
- Y una comunidad amplia donde encontrará ejemplos de todos tipo

# Inicialización

- Vaya a la ventana de comandos
- Inicialice el ambiente de trabajo, p.e.
> activate ai
- Seguido instale tensorflow, esta operación puede tomar varios minutos
> pip install tensorflow==1.15.0
- A veces salen errores en instalación de librerías, en mi caso, por incompatibilidad tuve que instalar tensorboard y tensorflow estimator
> pip install tensorboard==2.0.0 tensorflow-estimator==2.0.0
- Reinicie el kernel y carge nuevamente o cierre jupyter notebook y lancelo de nuevo.

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Verificando la versión de Tensorflow y su Instalación

In [2]:
import tensorflow as tf

tf.__version__

'1.15.0'

Si lo anterior está bien, entonce debe observar la versión instalada de tensorflow.

¿Porqué utilizaremos tensorflow 1.15.0 que es la última versión de tensorflow 1.x?, pues, para ver las diferencias entre el viejo framework y el nuevo, además que existen más ejemplos de tensorflow 1.x hasta el momento que de tensorflow 2.x, sin embargo se tomará en cuenta el nuevo framework a futuro.

# Creando una gráfica en Tensorflow

In [3]:
import tensorflow as tf

reset_graph()

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

Lo primero es iniciar una gráfica.

Estas líneas anteriores no ejecutan nada, es decir, no darán un resultado, las variables no han sido ni siquiera inicializadas a pesar que pareciera que sí.  Para evaluar lo anterior necesitamos construir una sesión y evaluar la función $f$. 

In [4]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [5]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
result

42

In [6]:
sess.close()

Luego de inicializada la gráfica:
- Realizamos la sesión
- Inicializamos las variables 'x' y 'y'
- Evaluamos el resultado
- Finalmente cerramos la sesión

In [7]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

result

42

Una mejor manera, o más pulcra/elegante de evaluar la sesión en Tensorflow es como el código anterior.

Las **x.initializer.run()** y **y.initializer.run()** crearán las asignaciones de las variables y **f.eval()** computará el resulatdo.

Como verá no cerramos la sesión, y esto se debe a que el 'snipet' de código anterior lo hace solo.

Compare los códigos de las dos últimas celdas donde aparece sess y verá su analogía.


In [8]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    
result

42

Otra manera más eficiente de inicializar las variables es por medio de **tf.global_variables_initializer()**.  También observe que no solo inicializa sino que crea el nodo.

In [9]:
init = tf.global_variables_initializer()

sess = tf.InteractiveSession()
init.run()
result = f.eval()

result

42

In [10]:
sess.close()

In [11]:
result

42

En resumen, tensorflow se divide en dos partes:
- La gráfica de computación
- Sección donde arrancan las funciones/computo

La prmera tiene que ver como se 'construye' la gráfica, relación de variables y la segunda tiene que ver en como se ejecuta esta gráfica.  Esto es mandatorio para su aplicación en tensorflow.

# Administrando las Gráficas

In [12]:
reset_graph()

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [13]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [14]:
x2.graph is tf.get_default_graph()

False

Los nodos creados se añaden a la gráfica principal, esto es una regla en tensorfow!

Si va a crear una nueva gráfica entonces debe utilizar **tf.reset_default_graph()** para que no se mezclen los nodos viejos y nuevos.

# Ciclo de Vida de un Nodo

In [15]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15

10
15


Como comentamos, en las sesiones se dan las inicializaciones y evaluaciones de variables.  Tensorflow lo hace cada ez que se evalua el nodo.

Al crear la sesión y evaluar la variable vemos que Tensorflow 'sabe' las dependencias de 'y' que depende de 'x' y así mismo 'z' también depende de 'x' u que 'x' depende de 'w'

In [16]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)  # 10
    print(z_val)  # 15

10
15


Una manear más eficiente de hacer esto es evaluando en cada corrida 'y' y 'z'.  Por el momento veremos que múltiples sesiones en Tensorflow no comparten variables, a diferrencia de cuando se hace entrenamiento distribuido que la información si se guarda en servidores (no en sesiones como se explica aquí).

# Regresión Lineal

En tensorflow las variables son declaradas antes de las sesiones.  Conocemos de programación que los arreglos pueden ser conjunto de variables, estos arreglos pueden ser unidimensionales, que también se les conoce como vectores o más de una dimensión por ejemplo 10x3 sería un arreglo de 10 filas 3 columnas, para un total de 30 elementos.

En tensorflow, los arreglos de más de dos dimensiónes se les llama tensores, los tensores como cualquier variable tienen un tipo (.dtypes) y forma (.shape).

## Utilizando la Ecuación Normal

In [17]:
import numpy as np
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
theta_value

array([[-3.7174683e+01],
       [ 4.3591911e-01],
       [ 9.3909204e-03],
       [-1.0637519e-01],
       [ 6.4145899e-01],
       [-4.1128196e-06],
       [-3.7799443e-03],
       [-4.2388692e-01],
       [-4.3728542e-01]], dtype=float32)

El código anterior hace lo que describimos a continuación:
- Reinicializamos el gráfico
- Descargamos el dataset de precios de casas
- Agregamos una característica inicial X0, llamada bias (recuerde otros laboratorios) a cada caraterística del dataset
- X es nuestra característica de entrada (.data) y la pasamos a un tensor
- y es nuestra característica de salida (.target), cambiamos la forma para procesarla en un tensor de 1 columna
- Finalmente hacemos operaciones matemáticas de matrices $\theta = (X^T*X)^{-1}*X^T*y$, recuerde laboratorio anterior 4
- Creamos una sesión y finalmente evaluamos $\theta$

La ventaja de esto versus hacerlo en numpy es la facilidad de aprovechar el uso del GPU, pero la versión instalada que poseemos solo usa el CPU, para que todos podamos correr la misma aplicación.  Si desea instalar tensorflow para usar el gpu existen unos requerimientos de paquetes de NVIDIA de CUDA que se deben cumplir, ver la documentasión para instalar tensorflow-gpu.

In [18]:
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


Comparación con numpy

El código que también se puede realizar utilizando numpy, que es más corto, pero como comentamos, la ventaja de Tensorflow vendrá despues en lo que es procesamiento parelelo y librerías avanzadas de Machine Learning / Deep Learning.

In [19]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


Comparación con Scikit-Learn

Puede conseguir el mismo resultado solo con las dos librerías anteriores, pero recomedamos el uso de frameworks avanzados como p.e. tensorflow que es muy popular o pytorch, que no se explicará en este curso.

## Implementando el Uso en Batch Gradient Descent

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [21]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


Para gradiente descendiente vamos a utilizar la función de standard scaler en scikit-learn (que también se puede realizar en tensorflow).

### Calculando los gradientes manualmente

In [22]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.71450055
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.55557173
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962904
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473


In [23]:
best_theta

array([[ 2.0685523 ],
       [ 0.8874027 ],
       [ 0.14401656],
       [-0.34770885],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.66145283],
       [-0.6375278 ]], dtype=float32)

Para explicar algunas cosas que no son obvias del código:
- **random_uniform()** crea un nodo que es el generador de números aleatorios
- **assign()** crea un nodo que asigna el valor a la variable.  Para implementar gradiente descendiente necesitamos actualizar $\theta$.  $\theta^{'} = \theta - \eta\nabla MSE(\theta)$
- Y finalmente se ejecutará muchas veces, 1000 ciclos del dataset (1 epoch = 1 vuelta del dataset)

### Utilizando autodiff

Same as above except for the `gradients = ...` line:

In [24]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [25]:
gradients = tf.gradients(mse, [theta])[0]

In [26]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.7145004
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.55557173
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962904
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473
Best theta:
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


Esta sería la manera de calcular gradiente descendiente con tensorflow.  

Creadas las variables y nodos vemos que calculamos el mse.

Seguido nuestra variable a derivar es $\theta$ por lo que en la función **tf.gradients()** la utilizamos en []. 

Finalmente asignamos esta variable utilizando la ecuación de gradiente descendiente.  Recuerde que aún no han sido evaluadas las condiciones.  Finalmente en la sesión es en donde realizamos las evaluacioens de esta función de gradiente descendiente e imprimimos el mejor valor de $\theta$

In [27]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

In [28]:
my_func(0.2, 0.3)

-0.21253923284754914

In [29]:
reset_graph()

a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.constant(0.0, name="z0")
for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)

grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

Let's compute the function at $a=0.2$ and $b=0.3$, and the partial derivatives at that point with regards to $a$ and with regards to $b$:

In [30]:
with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads))

-0.21253741
[-1.1388495, 0.19671395]


Lo anterior calcula los gradientes de una función con las constantes 'a' y 'b'.  Esta es la forma correcta y eficiente de hacerlo.  

- Primero definimos la función
- Seguido creamos un gráfico y utilizamos la función **tf.gradients()**
- Finalmente creados los nodos evaluamos y tenemos el resulatdo de tanto la función de evaluación de z como el gradiente de z

### Usando la función `GradientDescentOptimizer`

In [31]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [32]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [33]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.7145004
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.55557173
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962904
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473
Best theta:
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


El mismo código que se vio en la sección anterior ahora lo realizaremos con un optimizador que trae Tensorflow por defecto llamado GradientDescentOptimizer.  Así tensorflow tiene algunos optimizadores 'en caja' que podemos utilizar sin necesidad de hacer nuestra función de pérdida nosotros mismos.

Las líneas más importantes son las que reemplazan el optimizador y la función que ejecuta el optimzador, es decir las líneas
- optimizer
- training_op

Si queremos cambiar y probar otro optimizador solamente hay qeu cambiar el optimizador de la línea **optimizer**

### Utilizando `MomentumOptimizer`

In [34]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [35]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)

In [36]:
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [37]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Best theta:
[[ 2.068558  ]
 [ 0.8296286 ]
 [ 0.11875337]
 [-0.26554456]
 [ 0.3057109 ]
 [-0.00450251]
 [-0.03932662]
 [-0.89986444]
 [-0.87052065]]


Mismo que lo anterior, utiliza otro tipo de optimizador.

# Alimentando Datos en los algoritmos

## Placeholder nodes

In [38]:
reset_graph()

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)

[[6. 7. 8.]]


In [39]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


Los placeholder son herramientas buenas a la hora de entrenar algoritmos, pues estos solo guardan la información de momento.

Para crear un placeholder solamente tenemos que especificar el tipo de datos (mandatorio, de lo contrario dará un error) y opcionalmente la forma del tipo de datos (si esta no se especifica se entiende que tiene cualquier tamaño).

Lo nuevo aquí es el feed_dict.  Un diccionario permite agregar varios tipos de datos de entrada a en este caso nuestra función de evaluación.

## Implementando el `Mini-batch Gradient Descent`

In [40]:
n_epochs = 1000
learning_rate = 0.01

In [41]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [42]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [43]:
n_epochs = 10

In [44]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [45]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [46]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

Para implementar mini-batch gradiente descent, recuerde que, por ejemplo.  Si nuestro dataset tiene 100 conjuntos de datos y queremos implementar el mini-batch, digamos que son 4 mini-batch, serian 4 grupos de 25 en cada iteración tanto para nuestras características como para nuestras salidas.

- Creamos los placeholder 'X' y 'y'
- Definimos el batch size y el total del batch
- Fialmente realizamos una función que nos 'pagine' (paginar es una técnica web que no despliega todas las páginas de una sola vez, p.e. al buscar en google, solo se muestra un grupo de el total posible de páginas y debajo puede navegar a las siguientes o volver a la anterior), p.e. de la 0-100, 101-200, 201-300, etc.
- Finalmente en la sesión llamamos a la función de paginación del mini-batch y alimentamos al algoritmo de training que hace la optimización.

Estudie bien el código anterior pues es la base para hacer training en tensorflow!  Este es más o menos el esqueleto de como llamar un training set y entrenar un modelo!

# Guardando y Restaurando un Modelo

In [47]:
import os

if not os.path.exists('tfmodel'):
    os.mkdir('./tfmodel')
    print("Creating 'tfmodel' directory\n")
    
reset_graph()

n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "./tfmodel/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./tfmodel/my_model_final.ckpt")

Creating 'tfmodel' directory

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.7145004
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.55557173
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962904
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473


In [48]:
best_theta

array([[ 2.0685525 ],
       [ 0.8874027 ],
       [ 0.14401658],
       [-0.34770882],
       [ 0.36178368],
       [ 0.00393811],
       [-0.04269556],
       [-0.6614528 ],
       [-0.6375277 ]], dtype=float32)

Observe ahora que la única diferencia del código anterior al mini-batch gradient descent es que añadimos un nodo de **saver.save()** de esta manera podemos salvar el modelo y cargarlo o consumirlo luego en la aplicación.

Es bueno también no solo salvar el modelo al final sino salvar el modelo cada cierto tiempo.

In [49]:
with tf.Session() as sess:
    saver.restore(sess, "./tfmodel/my_model_final.ckpt")
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from ./tfmodel/my_model_final.ckpt


In [50]:
np.allclose(best_theta, best_theta_restored)

True

Podemos luego restaurar nuestro modelo 'totalmente' utilizando un nodo de **Saver** y la función **.restore()**

In [51]:
saver = tf.train.Saver({"weights": theta})

También podemos utilizar el nodo **Saver** para restaurar específicas acciones del modelo como por ejemplo los pesos.

In [52]:
reset_graph()
# notice that we start with an empty graph.

saver = tf.train.import_meta_graph("./tfmodel/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # not shown in the book

with tf.Session() as sess:
    saver.restore(sess, "./tfmodel/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from ./tfmodel/my_model_final.ckpt


Por defecto también Tensorflow salva la gráfica en un archivo de extensión **.meta**.  Para acceder al gráfico debemos utilizar la función **tf.train.import_meta_graph()** para restaurar la estructura del gráfico.  Esta función creada por el nodo saver restaura totalmente el gráfico y su estado es asignado a la función inicial de la gráfica.

In [53]:
np.allclose(best_theta, best_theta_restored)

True

A mas detalle.  Podemos cargar el gráfico con el **.meta** sin tener el código inicial de la aplicación y modificar el modelo a placer con los hiperparámetros.

# Visualizando la gráfica...
### Dentro de Jupyter Notebooks


Snipet de código inferior del gist de [Arthur Douillard](https://gist.github.com/arthurdouillard/621ed889291df51f80ef33059d882c53) nos permite utilizar Tensorboard dentro de jupyter notebook.

In [54]:
import tensorflow as tf
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32, width=800, height=600):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:{height}px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()), height=str(height))
  
    iframe = """
        <iframe seamless style="width:{width}px;height:{height}px;border:0" srcdoc="{code}"></iframe>
    """.format(code=code.replace('"', '&quot;'), width=width, height=str(height + 20))
    display(HTML(iframe))

In [55]:
show_graph(tf.get_default_graph(), max_const_size=32, width=800, height=600)

El siguiente código no funciona pues el servicio que se utiliza para servir tensorboard https://tensorboard.appspot.com/ (que también se utiliza con acceso a internet, está caído en el momento que lo utilicé, a raíz de esto usamos la función anterior que nos facilita tensorboard en jupyter) El código original fue escrito por Alex Mordvintsev en el tutorial de [DeepDream tutorial](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb).Por otro lado puede utilizar también esta herramienta: [tfgraphviz](https://github.com/akimach/tfgraphviz).

In [56]:
# No está funcionando

#from tensorflow_graph_in_jupyter import show_graph

#show_graph(tf.get_default_graph())

## Utilizando Tensorboard

In [57]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "./tflogs"

logdir = "{}/run-{}/".format(root_logdir, now)

Recapitulando:
- Empezamos utilizando Tensorflow creando variables, nodos y placeholders
- Utilizamos tensorflow y optimizadores para mini-batch gradient descent
- Ya sabemos guardar y cargar tanto los gráficos como los modelos

Para no usar print y verlo solo numericamente vamos a guardar para luego visualizar en tensorboard una 'bolsa' de datos.  Esto podemos hacerlo también por el navegador, pero como vimos lo hemos embebido en jupyter para mayor facilidad.

Esto es para facilitar el entendimiento del aprendizaje, aqui modificamos el programa un poco para saber a que tiempo corresponden esos datos y como a cual se relacionan.

In [58]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [59]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [60]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [61]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     # not shown

In [62]:
file_writer.close()

In [63]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255151],
       [-0.31211874],
       [ 0.38510373],
       [ 0.00434168],
       [-0.01232954],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

La única modificación realizada al código para guardar las 'bolsas' de datos son la línea **mse_summary** y **file_writter**.  La primera línea crea un nodo en la gráfica que evalua la función de MSE y la escribe en una cadena binaria llamda **summary**. 

No hay necesidad de validar el directorio si existe o no, **file_writer** creará el directorio si este no existe.  

Finalmente dentro de la sesión se evalua la función **mse_summary** pasandose a una variable y esta luego se inserta por medio de **file_writer** con su dato y el punto en que ocurrió.

Finalmente y siempre se cierra el archivo de datos.

Navegue a la carpeta de proyecto y verá que en tflogs se habrá creado el archivo.  Cada vez que corra esto se generará una 'bolsa' de datos con estampa de tiempos diferentes.

# Visualzando Tensorboard

Para visualizar Tensorboard.

- Abra una ventana de comandos
- Active el ambiente
- Navegue a la carpeta del proyecto
- Ingrese el comado 
> tensorboard --logdir tflogs
- Ignore los errores si no posee GPU y espera
- Abrir una pagina web luego que aparece el mensaje que puede lanzar tensorboard e ingrese:
> http://localhost:6006

Juegue con tensorboard

Observe que puede visualizar tanto la gráfica como el MSE del nodo como fue mejorando en el training.

#### Scalars
En scalars demore un rato y observe la progresión y como puede observar y modificar el portal para tener una mejor panorámica de los datos.

#### Graphs
Aquí se puede ver el flujo de los datos en el modelo.  Existen dos gráficas, una panorámica del modelo (izq) y una gráfica reducida del modelo (der).  Si hace doble click en 'gradients' observará que el nodo se expandirá y tendrá acceso a más visibilidad del modelo como va genrando eta gráfica.

# Alcances de Nombres [Name Scopes]

In [64]:
reset_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tflogs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

In [65]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

Name scopes es una manera cómoda  de trabajar agrupando más de una variable, en este ejemplo se alojan 'error' y 'mse' como 'loss' para tener una mejor idea de que simbolizan estas variables, esto es ventajoso a la hora de redes neuronales pues podemos confundirnos debido a que ellas tienen mucha información de entrada y/o salida a observar.

Para esto usamos la función **tf.name_scope()**

In [66]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [67]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.8030471 ]]


In [68]:
print(error.op.name)

loss/sub


In [69]:
print(mse.op.name)

loss/mse


Ahora los nodos de operación fueron 'tageados' con su nombre lo cual nos facilita encontrarlos en la gráfica.  Tienen el prefijo 'loss' y ahora son cada uno hijos de él agrupando de esta manera y visualizando mejor que simbolizan.

In [70]:
reset_graph()

a1 = tf.Variable(0, name="a")      # name == "a"
a2 = tf.Variable(0, name="a")      # name == "a_1"

with tf.name_scope("param"):       # name == "param"
    a3 = tf.Variable(0, name="a")  # name == "param/a"

with tf.name_scope("param"):       # name == "param_1"
    a4 = tf.Variable(0, name="a")  # name == "param_1/a"

for node in (a1, a2, a3, a4):
    print(node.op.name)

a
a_1
param/a
param_1/a


Más ejemplos de 'scope'

# Modularidad de Código

Veamos un ejemplo de lo que no se debe hacer:

In [71]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

El código anterior funciona, sin embargo, la línea **relu2** tiene un error, debemos de pasar la función de salida a la siguiente entrada, también es un código difícil de mantener.

In [72]:
reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [73]:
file_writer = tf.summary.FileWriter("tflogs/relu1", tf.get_default_graph())
file_writer.close()

La ecuación de ReLU es: $ h_{w,b}(X)=max(X*w+b,0)$

Lo mejor es escribir una función que evalue cada vez que se ejecuta ReLU.  Este generará para el gráfico los nodos:
- weights, weights_1, weights_2
- bias, bias_1, bias_2
- z y otros nodos

Visualice la gráfica generada en Tensorboard para entender el detalle.

In [74]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    # not shown
        b = tf.Variable(0.0, name="bias")                             # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                      # not shown
        return tf.maximum(z, 0., name="max")                          # not shown

In [75]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("tflogs/relu2", tf.get_default_graph())
file_writer.close()

Lo más reconedado es usar namescopes de esta manera como se muestra anteriormente, es decir, utilizando name scopes.


## Variables Compartidas

In [76]:
reset_graph()

def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

Una opción para 'controlar' la variable 'threshold' es:
- Crear la variable
- Pasarla luego como un parámetro de la función de ReLU necesitada
- Finalmente pasarla a la salida

Lo anterior funciona bien para cuando es una sola variable, ahora veamos el caso cuando es multivariable.

In [77]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, relu.threshold, name="max")

In [78]:
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

Otra opción más elegante es crear un atributo dentro de la función de ReLU, si no la posee y la necesita, crearla.

In [79]:
reset_graph()


with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))

Lo anterior es la función recomendada por Tensorflow para realizar asignación y manipulación de variables compartidas.

Primero debemos crear una **get_variable()** esta tiene de entradas el nombre, la forma y un inicializador.  La idea de esto es que si la variable no existe se crea y si existe se utiliza con los valores actuales que posee.

Esto creará la variable 'relu/threshold'.

Si volvemos a utilizar la variable ya creada nos dará un error, pruebelo comentando la línea de **reset_graph()** y ejecutelo nuevamente.

In [80]:
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

El código reutilizará la variable 'relu/treshold' y si no existe mandará un mesaje de error

In [81]:
with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

Diferentes maneras de crear las variables con **variable_scope()**

In [82]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

In [83]:
file_writer = tf.summary.FileWriter("tflogs/relu6", tf.get_default_graph())
file_writer.close()

El código anterior primero define una función de relu(), crea la variable 'relu/threshold' como escalar inicialziada en 0.0 y construirá 5 funciones de relu que utilizan la variable 'relu/threshold'.

In [84]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu"):
        threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("", default_name="") as scope:
    first_relu = relu(X)     # create the shared variable
    scope.reuse_variables()  # then reuse it
    relus = [first_relu] + [relu(X) for i in range(4)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("tflogs/relu8", tf.get_default_graph())
file_writer.close()

In [85]:
reset_graph()

def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [86]:
file_writer = tf.summary.FileWriter("tflogs/relu9", tf.get_default_graph())
file_writer.close()

# Material Extra

In [87]:
reset_graph()

with tf.variable_scope("my_scope"):
    x0 = tf.get_variable("x", shape=(), initializer=tf.constant_initializer(0.))
    x1 = tf.Variable(0., name="x")
    x2 = tf.Variable(0., name="x")

with tf.variable_scope("my_scope", reuse=True):
    x3 = tf.get_variable("x")
    x4 = tf.Variable(0., name="x")

with tf.variable_scope("", default_name="", reuse=True):
    x5 = tf.get_variable("my_scope/x")

print("x0:", x0.op.name)
print("x1:", x1.op.name)
print("x2:", x2.op.name)
print("x3:", x3.op.name)
print("x4:", x4.op.name)
print("x5:", x5.op.name)
print(x0 is x3 and x3 is x5)

x0: my_scope/x
x1: my_scope/x_1
x2: my_scope/x_2
x3: my_scope/x
x4: my_scope_1/x
x5: my_scope/x
True


El primer bloque crea una varaible x0 llamada 'my_scope/x'.  Esta actua como un nombre regular y asigna las otras variables 'x1' y 'x2'con el prefijo 'my_scope/' tambien. Tensorflow para diferenciarlas les añade un _1 y _2.

El segundo bloque utiliza nuevamente el scope, per ahora 'x3' es 'my_scope/x' y 'x4' . Para las varaibles que ya han sido creadas por 'get_variable' tienen el mismo alcance, pero para la nueva variable crea un alcance diferente 'my_scope_1/x'.

Este último bloque tiene 'get_variable' así que como 'variable_scope' tiene True de argumento reusa la variable ya creada 'my_scope/x'.

## Cadena de Caracteres

In [88]:
reset_graph()

text = np.array("Do you want some café?".split())
text_tensor = tf.constant(text)

with tf.Session() as sess:
    print(text_tensor.eval())

[b'Do' b'you' b'want' b'some' b'caf\xc3\xa9?']


# Laboratorio

Implemente un modelo en tensorflow 1.x (100%)
- Carge el dataset de [Titanic](https://www.tensorflow.org/datasets/catalog/titanic).  Puede hacerlo con [Tensorflow Datasets](https://www.tensorflow.org/datasets) - 10%
- Defina y utilice una grafica de función de regresión logística (esta función ya está implementada en tensorflow) - 10%
- Explore el dataset (haga una gráfica que le ayude a comprender, p.e., cuantas personas menores a 18 años sobrevivieron y cuales de estos eran niños y cuales niñas, es un ejemplo) - 20%
- Divida el dataset en 80/20. - 10%
- Entrene y prediga cuales pasajeros fueron los sobrevivientes - 10%
- Guarde el checkpoint usando Saver - 10%
- Restaure el checkpoint - 10%
- Añada la función summary para visualizar en tensorboard -10%
- Trate de modificar algunos hiperparámetros para mejorar el training - 10%